In [8]:
import os
import re
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics.pairwise import cosine_similarity

# ============================================================
# 0. FUNZIONI MANCANTI (aggiunte qui!)
# ============================================================

def normalize_percentage(v):
    """
    Normalizza il valore percentuale:
    - Se 0 < v < 1 → frazione → v*100
    - Se v ≈ 1 → 1%
    - Se v >= 3 → già percentuale
    """
    try:
        v = float(v)
    except:
        return None

    if v == 0:
        return 0

    if 0 < v < 0.8:
        return v * 100

    return v

# ============================
# 1. Caricamento indice + dataset stats
# ============================

MODELS_DIR = "../data/models"
DATA_CLEAN_PATH = "../data/drafted_cleaned.csv"

VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

vectorizer = joblib.load(VECTORIZER_PATH)
tfidf_matrix = joblib.load(MATRIX_PATH)
metadata_df = pd.read_csv(META_PATH)

print("Indice caricato.")
print("TF-IDF matrix shape:", tfidf_matrix.shape)
print("Metadata shape:", metadata_df.shape)

# Carico dataset completo con le stats per spiegazioni
full_df = pd.read_csv(DATA_CLEAN_PATH)
print("Dataset stats shape:", full_df.shape)

# Assumo che player_id sia l'indice originale usato in Step A
if "player_id" not in metadata_df.columns:
    # Se non c'è, lo ricreo coerente
    metadata_df["player_id"] = metadata_df.index.astype(int)

# ============================
# 2. Utility di base
# ============================

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

def safe_get(row, col, default=None):
    if col in row and not pd.isna(row[col]):
        return row[col]
    return default

def clean_stat(value):
    """
    Converte -1 e -100 (valori sentinella del dataset) in None.
    Lascia invariati 0, 0.0 o altri numeri validi.
    """
    if value is None or pd.isna(value):
        return None
    try:
        v = float(value)
    except:
        return None

    # Valori sentinella del tuo dataset
    if v in (-1, -100):
        return None

    return v

# ============================
# 3. Stesse regole semantiche Step C
# ============================



SKILL_PATTERNS = {
    "shooting_3": [
        "tiratore da 3", "ottime percentuali da 3", "buon tiratore da 3",
        "da 3 punti", "tripla", "3 punti"
    ],
    "ft": [
        "buon ft", "ottimo ft", "buon tiratore di liberi", "ottimo tiratore di liberi",
        "buon ft%", "tiri liberi", "liberi"
    ],
    "reb": [
        "rimbalzista", "forte rimbalzista", "buon rimbalzista", "rimbalzi",
        "dominante a rimbalzo", "forte a rimbalzo"
    ],
    "playmaking": [
        "playmaker", "buon passatore", "assist", "creatore di gioco",
        "regista", "fa girare la squadra"
    ],
    "scorer": [
        "realizzatore", "scorer", "tanti punti", "tanti punti a partita",
        "prima opzione offensiva"
    ]
}

SEMANTIC_EXPANSIONS = {
    "shooting_3": "ottimo tiratore da 3 punti tiro da 3 3P% buon tiratore da 3",
    "ft":         "buon tiratore di tiri liberi FT% tiri liberi eccellente tiratore di tiri liberi",
    "reb":        "forte rimbalzista rimbalzi buon rimbalzista",
    "playmaking": "ottimo playmaker e passatore buon passatore assist",
    "scorer":     "segna in media molti punti punti a partita realizzatore"
}

SEMANTIC_WEIGHTS = {

    "shooting_3": 4,
    "ft":         3,
    "reb":        3,
    "reb_off":    4,
    "playmaking": 3,
    "defense":    3,
    "scorer":     3
}

#Status del giocatore, se è in attività o meno
STATUS_PATTERNS = {
    "Active": ["in attività", "ancora in attività", "ancora gioca", "sta ancora giocando"],
    "Retired": ["ritirato", "non gioca più", "ha smesso", "in pensione"]
}

# ============================
# 4. Parsing query -> intenti
# ============================

def parse_query_intents(raw_query: str):
    q = normalize_text(raw_query)
    detected_status = None
    detected_skills = set()

    # Stato (attivo / ritirato)
    for status, patterns in STATUS_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_status = status
                break
        if detected_status is not None:
            break

    #Skill
    for skill, patterns in SKILL_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_skills.add(skill)
                break

    intents = {
        "normalized_query": q,
        "skills": sorted(list(detected_skills)),
        "status": detected_status if detected_status is not None else None

    }
    return intents

def build_weighted_query(raw_query: str, intents: dict) -> str:
    base = intents.get("normalized_query", normalize_text(raw_query))
    boosted_parts = [base]

    for role in intents.get("roles", []):
        if role in SEMANTIC_EXPANSIONS:
            exp = SEMANTIC_EXPANSIONS[role]
            w = SEMANTIC_WEIGHTS.get(role, 1)
            boosted_parts.append((" " + exp) * w)

    for skill in intents.get("skills", []):
        if skill in SEMANTIC_EXPANSIONS:
            exp = SEMANTIC_EXPANSIONS[skill]
            w = SEMANTIC_WEIGHTS.get(skill, 1)
            boosted_parts.append((" " + exp) * w)

    weighted_query = " ".join(boosted_parts)
    return weighted_query.strip()

# ============================
# 5. Ricerca con query pesata (come Step C)
# ============================

def search_players_scout(query: str, top_k: int = 10):
    intents = parse_query_intents(query)
    weighted_query = build_weighted_query(query, intents)

    df = metadata_df.copy()

    # -------- FILTRO STATUS PRIMA DELLA RICERCA -----------
    if intents["status"] is not None:
        df = df[df["Status"] == intents["status"]]

    # se vuoto → backup su tutto il dataset
    if df.empty:
        df = metadata_df.copy()

    # FILTRO matrix TF-IDF
    valid_idx = df.index.tolist()
    tfidf_sub = tfidf_matrix[valid_idx]

    # Similarità
    q_vec = vectorizer.transform([weighted_query])
    sims = cosine_similarity(q_vec, tfidf_sub).ravel()

    df = df.copy()
    df["similarity"] = sims
    df = df.sort_values("similarity", ascending=False).head(top_k)

    df.attrs["intents"] = intents
    df.attrs["weighted_query"] = weighted_query
    return df



# ============================
# 6. Funzioni per spiegazioni (VERSIONE CORRETTA)
# ============================



def format_percent_it(value, decimals=1):
    v = normalize_percentage(value)
    if v is None:
        return None
    s = f"{v:.{decimals}f}".replace(".", ",")
    return s + "%"


def format_number_it(value, decimals=1):
    """
    Formatta un numero con virgola italiana: 6.5 -> '6,5'
    """
    if value is None or pd.isna(value):
        return None
    try:
        v = float(value)
    except (TypeError, ValueError):
        return None
    s = f"{v:.{decimals}f}"
    return s.replace(".", ",")

def bucket_3p(pct):
    if pct is None or pd.isna(pct):
        return None
    try:
        v = float(pct)
    except (TypeError, ValueError):
        return None
    if v <= 1:
        v *= 100
    if v >= 40:
        return "ottimo tiratore da 3 punti"
    elif v >= 35:
        return "buon tiratore da 3 punti"
    elif v >= 30:
        return "tiratore da 3 punti discreto"
    else:
        return "tiratore da 3 punti poco affidabile"

def bucket_ft(pct):
    if pct is None or pd.isna(pct):
        return None
    try:
        v = float(pct)
    except (TypeError, ValueError):
        return None
    if v <= 1:
        v *= 100
    if v >= 85:
        return "eccellente tiratore di tiri liberi"
    elif v >= 75:
        return "buon tiratore di tiri liberi"
    else:
        return "tiratore di liberi migliorabile"

def build_explanation(player_row, intents):
    """
    Costruisce una breve spiegazione del perché il giocatore matcha la query.
    Usa:
      - ruolo (Pos)
      - stats numeriche (3P%, FT%, TRB per partita, ORB per partita, AST, DWS, PTS)
      - skill richieste
    """
    skills = intents.get("skills", [])
    name = player_row.get("Player", "Il giocatore")
    reasons = [f"{name} è stato selezionato perché:"]

    status_val = player_row.get("Status", None)
    if status_val == "Active":
        reasons.append("- è ancora in attività in NBA.")
    elif status_val == "Retired":
        reasons.append("- è un giocatore ritirato dalla NBA.")

    # --- Percentuali tiro ---
    p3 = clean_stat(safe_get(player_row, "3P%") or safe_get(player_row, "ThreeP"))
    ft = safe_get(player_row, "FT%") or safe_get(player_row, "FT")

    skills = intents.get("skills", [])

    # Shooting da 3
    if "shooting_3" in skills and p3 is not None:
        desc_3p = bucket_3p(p3)
        p3_str = format_percent_it(p3)
        if desc_3p and p3_str:
            reasons.append(f"- {desc_3p} ({p3_str} da tre).")

    # Tiri liberi
    if "ft" in skills and ft is not None:
        desc_ft = bucket_ft(ft)
        ft_str = format_percent_it(ft)
        if desc_ft and ft_str:
            reasons.append(f"- {desc_ft} ({ft_str} ai liberi).")

    # --- Rimbalzi: usa per-partita se disponibile ---
    # Proviamo prima colonne "per game". Tu mi hai detto che i per-game sono in AST.1,
    # quindi la includo esplicitamente tra le possibilità.
    trb_pg = None

    for col in ["TRB_per_game", "TRB_pg", "TRB.1", "AST.1"]:
        if col in player_row.index and not pd.isna(player_row[col]):
            trb_pg = player_row[col]
            break

    # Se non trovo rimbalzi per partita, provo a derivarli da TRB totale / G
    if trb_pg is None:
        trb_total = safe_get(player_row, "TRB") or safe_get(player_row, "REB")
        games = safe_get(player_row, "G")
        try:
            if trb_total is not None and games not in (None, 0, np.nan):
                trb_pg = float(trb_total) / float(games)
        except:
            trb_pg = None

    # Rimbalzi totali offensivi per partita (se hai una colonna apposta)
    orb_pg = None
    for col in ["ORB_per_game", "ORB_pg"]:
        if col in player_row.index and not pd.isna(player_row[col]):
            orb_pg = player_row[col]
            break
    if orb_pg is None:
        # fallback: ORB totale / G
        orb_total = safe_get(player_row, "ORB")
        games = safe_get(player_row, "G")
        try:
            if orb_total is not None and games not in (None, 0, np.nan):
                orb_pg = float(orb_total) / float(games)
        except:
            orb_pg = None

    # Rimbalzi generali
    if "reb" in skills and trb_pg is not None:
        try:
            v = float(trb_pg)
            v_str = format_number_it(v)
            if v >= 9:
                reasons.append(f"- È un forte rimbalzista ({v_str} rimbalzi a partita).")
            elif v >= 6:
                reasons.append(f"- È un buon rimbalzista ({v_str} rimbalzi a partita).")
        except:
            pass

    # --- Playmaking ---
    ast = safe_get(player_row, "AST")  # qui di solito è AST per partita
    if "playmaking" in skills and ast is not None:
        try:
            v = float(ast)
            v_str = format_number_it(v)
            if v >= 7:
                reasons.append(f"- Ottimo playmaker e passatore ({v_str} assist a partita).")
            elif v >= 4:
                reasons.append(f"- Buon passatore ({v_str} assist a partita).")
        except:
            pass

    # --- Scorer ---
    pts = safe_get(player_row, "PTS") or safe_get(player_row, "PTS_per_game")
    if "scorer" in skills and pts is not None:
        try:
            v = float(pts)
            v_str = format_number_it(v)
            if v >= 20:
                reasons.append(f"- Realizzatore di alto livello ({v_str} punti a partita).")
            elif v >= 15:
                reasons.append(f"- Buon realizzatore ({v_str} punti a partita).")
        except:
            pass

    # Fallback se non ho ragioni specifiche
    if len(reasons) <= 1:
        reasons.append("- Il suo profilo testuale è complessivamente molto simile alla descrizione richiesta.")

    explanation = "\n".join(reasons)
    return explanation

# ============================
# 7. Funzione principale: ranking + spiegazioni
# ============================

def rank_and_explain(query, top_k=5):
    base = search_players_scout(query, top_k)
    intents = base.attrs["intents"]

    # Merge con il dataset completo
    if "player_id" in base.columns and "player_id" in full_df.columns:
        merged = base.merge(full_df, on="player_id", how="left")
    else:
        merged = base.merge(full_df, on="Player", how="left")

    # FILTRO SU STATO (Active / Retired) SE RICHIESTO
    status_req = intents.get("status", None)  # "Active" / "Retired" / None
    if status_req is not None and "Status" in merged.columns:
        merged = merged[merged["Status"] == status_req]

    merged["explanation"] = merged.apply(lambda r: build_explanation(r, intents), axis=1)

    return merged, intents


# ============================
# 8. Test
# ============================

test_query = (
    "Cerco un tiratore con ottime percentuali da 3,  "
    "forte rimbalzista e che sia ancora in attività."
)

results, intents = rank_and_explain(test_query, top_k=10)

print("Query originale:")
print(test_query)
print("\nIntenti interpretati:")
print(intents)

pd.set_option("display.max_colwidth", 200)

print("\nTOP risultati con spiegazioni (one-line):")
if "explanation" in results.columns:
    results["explanation_one_line"] = results["explanation"].str.replace("\n", " ")
    cols_to_show = [c for c in ["Player", "DraftYear", "Pick", "similarity", "explanation_one_line"] if c in results.columns]
    display(results[cols_to_show])
else:
    cols_to_show = [c for c in ["Player", "DraftYear", "Pick", "similarity"] if c in results.columns]
    display(results[cols_to_show])



Indice caricato.
TF-IDF matrix shape: (8323, 21993)
Metadata shape: (8323, 7)
Dataset stats shape: (8323, 36)
Query originale:
Cerco un tiratore con ottime percentuali da 3,  forte rimbalzista e che sia ancora in attività.

Intenti interpretati:
{'normalized_query': 'cerco un tiratore con ottime percentuali da 3, forte rimbalzista e che sia ancora in attività.', 'skills': ['reb', 'shooting_3'], 'status': 'Active'}

TOP risultati con spiegazioni (one-line):


,Player,similarity,explanation_one_line
0,Antonio Reeves,0.231046,"Antonio Reeves è stato selezionato perché: - ottimo tiratore da 3 punti (40,8% da tre)."
1,Doug McDermott,0.210512,"Doug McDermott è stato selezionato perché: - ottimo tiratore da 3 punti (41,0% da tre)."
2,Quinten Post,0.203996,"Quinten Post è stato selezionato perché: - ottimo tiratore da 3 punti (44,0% da tre)."
3,Ajay Mitchell,0.200391,"Ajay Mitchell è stato selezionato perché: - ottimo tiratore da 3 punti (43,1% da tre)."
4,Karl-Anthony Towns,0.172858,"Karl-Anthony Towns è stato selezionato perché: - ottimo tiratore da 3 punti (40,1% da tre). - È un forte rimbalzista (11,1 rimbalzi a partita)."
5,Payton Pritchard,0.165911,"Payton Pritchard è stato selezionato perché: - ottimo tiratore da 3 punti (40,3% da tre)."
6,Dariq Whitehead,0.163292,"Dariq Whitehead è stato selezionato perché: - ottimo tiratore da 3 punti (46,2% da tre)."
7,Luke Kennard,0.162714,"Luke Kennard è stato selezionato perché: - ottimo tiratore da 3 punti (43,9% da tre)."
8,Grayson Allen,0.162005,"Grayson Allen è stato selezionato perché: - ottimo tiratore da 3 punti (41,5% da tre)."
9,Jakob Poeltl,0.151795,"Jakob Poeltl è stato selezionato perché: - ottimo tiratore da 3 punti (42,9% da tre). - È un buon rimbalzista (7,0 rimbalzi a partita)."


In [7]:
print(metadata_df.columns)
print(full_df.columns)


Index(['player_id', 'Player', 'DraftYear', 'Pick', 'PickBand', 'Status',
       'text_profile'],
      dtype='object')
Index(['DraftYear', 'Pick', 'Tm', 'Player', 'College', 'Seasons', 'Games',
       'MP', 'PTS', 'TRB', 'AST', 'FG%', '3P%', 'FT%', 'MP.1', 'PTS.1',
       'TRB.1', 'AST.1', 'WS', 'WS/48', 'BPM', 'VORP', 'Forfeited', 'PickBand',
       'Debut', 'WS_available', 'WS/48_available', 'BPM_available',
       'VORP_available', 'MP_available', 'TRB_available', 'FG%_available',
       'FT%_available', 'MP.1_available', 'TRB.1_available', 'Status'],
      dtype='object')


In [14]:
print("metadata_df columns:", metadata_df.columns.tolist())
print("full_df columns:", full_df.columns.tolist())
print("results columns:", results.columns.tolist())
print(results.head())


metadata_df columns: ['player_id', 'Player', 'DraftYear', 'Pick', 'PickBand', 'Status', 'text_profile']
full_df columns: ['DraftYear', 'Pick', 'Tm', 'Player', 'College', 'Seasons', 'Games', 'MP', 'PTS', 'TRB', 'AST', 'FG%', '3P%', 'FT%', 'MP.1', 'PTS.1', 'TRB.1', 'AST.1', 'WS', 'WS/48', 'BPM', 'VORP', 'Forfeited', 'PickBand', 'Debut', 'WS_available', 'WS/48_available', 'BPM_available', 'VORP_available', 'MP_available', 'TRB_available', 'FG%_available', 'FT%_available', 'MP.1_available', 'TRB.1_available', 'Status', '3P_norm']
results columns: ['player_id', 'Player', 'DraftYear_x', 'Pick_x', 'PickBand_x', 'Status_x', 'text_profile', 'similarity', 'DraftYear_y', 'Pick_y', 'Tm', 'College', 'Seasons', 'Games', 'MP', 'PTS', 'TRB', 'AST', 'FG%', '3P%', 'FT%', 'MP.1', 'PTS.1', 'TRB.1', 'AST.1', 'WS', 'WS/48', 'BPM', 'VORP', 'Forfeited', 'PickBand_y', 'Debut', 'WS_available', 'WS/48_available', 'BPM_available', 'VORP_available', 'MP_available', 'TRB_available', 'FG%_available', 'FT%_available'